In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, MaxPooling2D
from keras.optimizers import Adam
import cv2
import matplotlib.pyplot as plt

In [ ]:
x = np.load('Datasets/X-c23-balanced.npy')
y = np.load('Datasets/y-c23-balanced.npy')
p = np.random.permutation(x.shape[0])

x = x[p]
y = y[p]

In [ ]:
x_test = x[:2000]
y_test = y[:2000]

x_train = x[2000:]
y_train = y[2000:]

In [ ]:
model = Sequential()

model.add(Conv2D(8, kernel_size=3, activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(16, kernel_size=5, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, kernel_size=5, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, kernel_size=5, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x, y, validation_data=(x_test, y_test), epochs=500)

In [ ]:
with open('model.json', 'w') as json_file:
    json_file.write(model.to_json())
    model.save_weights('model.h5')

In [ ]:
with open('model.json', 'r') as json_file:
    model = keras.models.model_from_json(json_file.read())
    model.load_weights('model.h5')

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
model.evaluate(x_test, y_test)